In [1]:
import os
import torch
import matplotlib.pyplot as plt
import sys
from tqdm import tqdm
sys.path.insert(0, './src')
%matplotlib inline


from metrics import NDCG_binary_at_k_batch, Recall_at_k_batch
from models import MultiVAE, MultiDAE, Multi_our_VAE, MultiHoffmanVAE, Multi_ourHoffman_VAE
from training import train_model
from data import Dataset
from args import get_args
import numpy as np
import pandas as pd

import pdb

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import ncvis




device = 'cuda:1'

class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

In [2]:
args = dotdict({})
args.train_batch_size = 500
args.val_batch_size = 2000
args.data = 'ml20m'
args.device = device

args.learning_rate = 1e-4
args.n_epoches = 200
args.l2_coeff = 0.01 / args.train_batch_size
args.print_info_ = 1

args.annealing = True

if args.annealing:
    args.total_anneal_steps = 200000
    args.anneal_cap = 0.2
else:
    args.total_anneal_steps = 0
    args.anneal_cap = 1.

device_zero = torch.tensor(0., device=device, dtype=torch.float32)
device_one = torch.tensor(1., device=device, dtype=torch.float32)
std_normal = torch.distributions.Normal(loc=device_zero,
                                    scale=device_one)
    
dataset = Dataset(args, data_dir='./data/')

In [3]:
models = [model for model in sorted(os.listdir('./models')) if model.startswith('best') ]
for m in models:
    print('\'' + m + '\',' )

'best_model_MultiHoffmanVAE_K_1_N_1_learnreverse_False_anneal_False_lrdec_0.001_lrenc_None.pt',
'best_model_MultiHoffmanVAE_K_1_N_1_learnreverse_False_anneal_True_lrdec_0.001_lrenc_None.pt',
'best_model_MultiHoffmanVAE_K_2_N_2_learnreverse_False_anneal_False_lrdec_0.001_lrenc_None.pt',
'best_model_MultiHoffmanVAE_K_2_N_2_learnreverse_False_anneal_True_lrdec_0.001_lrenc_None.pt',
'best_model_MultiHoffmanVAE_K_3_N_1_learnreverse_False_anneal_False_lrdec_0.0003_lrenc_0.001.pt',
'best_model_MultiHoffmanVAE_K_3_N_1_learnreverse_False_anneal_False_lrdec_3e-05_lrenc_0.001.pt',
'best_model_MultiHoffmanVAE_K_3_N_1_learnreverse_False_anneal_True_lrdec_0.0003_lrenc_0.001.pt',
'best_model_MultiHoffmanVAE_K_3_N_1_learnreverse_False_anneal_True_lrdec_3e-05_lrenc_0.001.pt',
'best_model_MultiHoffmanVAE_K_3_N_3_learnreverse_False_anneal_False_lrdec_0.001_lrenc_None.pt',
'best_model_MultiHoffmanVAE_K_3_N_3_learnreverse_False_anneal_True_lrdec_0.001_lrenc_None.pt',
'best_model_MultiHoffmanVAE_K_5_N_5_lea

In [4]:
# models = [
# 'best_model_MultiHoffmanVAE_K_1_N_1_learnreverse_False_anneal_False_lrdec_0.001_lrenc_None.pt',
# 'best_model_MultiHoffmanVAE_K_1_N_1_learnreverse_False_anneal_True_lrdec_0.001_lrenc_None.pt',
# 'best_model_MultiHoffmanVAE_K_2_N_2_learnreverse_False_anneal_False_lrdec_0.001_lrenc_None.pt',
# 'best_model_MultiHoffmanVAE_K_2_N_2_learnreverse_False_anneal_True_lrdec_0.001_lrenc_None.pt',
# 'best_model_MultiHoffmanVAE_K_3_N_3_learnreverse_False_anneal_False_lrdec_0.001_lrenc_None.pt',
# 'best_model_MultiHoffmanVAE_K_3_N_3_learnreverse_False_anneal_True_lrdec_0.001_lrenc_None.pt',
# 'best_model_MultiHoffmanVAE_K_5_N_5_learnreverse_False_anneal_False_lrdec_0.001_lrenc_None.pt',
# 'best_model_MultiHoffmanVAE_K_5_N_5_learnreverse_False_anneal_True_lrdec_0.001_lrenc_None.pt',
# 'best_model_MultiVAE_K_None_N_None_learnreverse_False_anneal_False_lrdec_0.001_lrenc_None.pt',
# 'best_model_MultiVAE_K_None_N_None_learnreverse_False_anneal_True_lrdec_0.001_lrenc_None.pt',
# 'best_model_MultiVAE_K_None_N_None_learnreverse_False_anneal_True_lrdec_3e-05_lrenc_0.001.pt',
# 'best_model_Multi_ourHoffman_VAE_K_1_N_1_learnreverse_False_anneal_False_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_ourHoffman_VAE_K_1_N_1_learnreverse_False_anneal_True_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_ourHoffman_VAE_K_1_N_1_learnreverse_True_anneal_False_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_ourHoffman_VAE_K_1_N_1_learnreverse_True_anneal_True_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_ourHoffman_VAE_K_1_N_2_learnreverse_False_anneal_False_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_ourHoffman_VAE_K_1_N_2_learnreverse_False_anneal_True_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_ourHoffman_VAE_K_1_N_2_learnreverse_True_anneal_False_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_ourHoffman_VAE_K_1_N_2_learnreverse_True_anneal_True_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_ourHoffman_VAE_K_1_N_3_learnreverse_False_anneal_False_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_ourHoffman_VAE_K_1_N_3_learnreverse_False_anneal_True_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_ourHoffman_VAE_K_1_N_3_learnreverse_True_anneal_False_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_ourHoffman_VAE_K_1_N_3_learnreverse_True_anneal_True_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_ourHoffman_VAE_K_1_N_5_learnreverse_False_anneal_False_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_ourHoffman_VAE_K_1_N_5_learnreverse_False_anneal_True_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_ourHoffman_VAE_K_1_N_5_learnreverse_True_anneal_False_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_ourHoffman_VAE_K_1_N_5_learnreverse_True_anneal_True_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_ourHoffman_VAE_K_2_N_1_learnreverse_False_anneal_False_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_ourHoffman_VAE_K_2_N_1_learnreverse_False_anneal_True_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_ourHoffman_VAE_K_2_N_1_learnreverse_True_anneal_False_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_ourHoffman_VAE_K_2_N_1_learnreverse_True_anneal_True_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_ourHoffman_VAE_K_2_N_2_learnreverse_False_anneal_False_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_ourHoffman_VAE_K_2_N_2_learnreverse_False_anneal_True_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_ourHoffman_VAE_K_2_N_2_learnreverse_True_anneal_False_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_ourHoffman_VAE_K_2_N_2_learnreverse_True_anneal_True_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_ourHoffman_VAE_K_3_N_1_learnreverse_False_anneal_False_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_ourHoffman_VAE_K_3_N_1_learnreverse_False_anneal_True_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_ourHoffman_VAE_K_3_N_1_learnreverse_True_anneal_False_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_ourHoffman_VAE_K_3_N_1_learnreverse_True_anneal_True_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_ourHoffman_VAE_K_3_N_3_learnreverse_False_anneal_False_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_ourHoffman_VAE_K_3_N_3_learnreverse_False_anneal_True_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_ourHoffman_VAE_K_3_N_3_learnreverse_True_anneal_False_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_ourHoffman_VAE_K_3_N_3_learnreverse_True_anneal_True_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_ourHoffman_VAE_K_5_N_1_learnreverse_False_anneal_False_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_ourHoffman_VAE_K_5_N_1_learnreverse_False_anneal_True_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_ourHoffman_VAE_K_5_N_1_learnreverse_True_anneal_False_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_ourHoffman_VAE_K_5_N_1_learnreverse_True_anneal_True_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_ourHoffman_VAE_K_5_N_5_learnreverse_False_anneal_False_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_ourHoffman_VAE_K_5_N_5_learnreverse_False_anneal_True_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_ourHoffman_VAE_K_5_N_5_learnreverse_True_anneal_False_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_ourHoffman_VAE_K_5_N_5_learnreverse_True_anneal_True_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_our_VAE_K_1_N_1_learnreverse_False_anneal_False_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_our_VAE_K_1_N_1_learnreverse_False_anneal_True_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_our_VAE_K_1_N_1_learnreverse_True_anneal_False_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_our_VAE_K_1_N_1_learnreverse_True_anneal_True_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_our_VAE_K_1_N_2_learnreverse_False_anneal_False_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_our_VAE_K_1_N_2_learnreverse_False_anneal_True_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_our_VAE_K_1_N_2_learnreverse_True_anneal_False_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_our_VAE_K_1_N_2_learnreverse_True_anneal_True_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_our_VAE_K_1_N_3_learnreverse_False_anneal_False_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_our_VAE_K_1_N_3_learnreverse_False_anneal_True_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_our_VAE_K_1_N_3_learnreverse_True_anneal_False_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_our_VAE_K_1_N_3_learnreverse_True_anneal_True_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_our_VAE_K_1_N_5_learnreverse_False_anneal_False_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_our_VAE_K_1_N_5_learnreverse_False_anneal_True_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_our_VAE_K_1_N_5_learnreverse_True_anneal_False_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_our_VAE_K_1_N_5_learnreverse_True_anneal_True_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_our_VAE_K_2_N_1_learnreverse_False_anneal_False_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_our_VAE_K_2_N_1_learnreverse_False_anneal_True_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_our_VAE_K_2_N_1_learnreverse_True_anneal_False_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_our_VAE_K_2_N_1_learnreverse_True_anneal_True_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_our_VAE_K_2_N_2_learnreverse_False_anneal_False_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_our_VAE_K_2_N_2_learnreverse_False_anneal_True_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_our_VAE_K_2_N_2_learnreverse_True_anneal_False_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_our_VAE_K_2_N_2_learnreverse_True_anneal_True_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_our_VAE_K_3_N_1_learnreverse_False_anneal_False_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_our_VAE_K_3_N_1_learnreverse_False_anneal_True_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_our_VAE_K_3_N_1_learnreverse_True_anneal_False_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_our_VAE_K_3_N_1_learnreverse_True_anneal_True_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_our_VAE_K_3_N_3_learnreverse_False_anneal_False_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_our_VAE_K_3_N_3_learnreverse_False_anneal_True_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_our_VAE_K_3_N_3_learnreverse_True_anneal_False_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_our_VAE_K_3_N_3_learnreverse_True_anneal_True_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_our_VAE_K_5_N_1_learnreverse_False_anneal_False_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_our_VAE_K_5_N_1_learnreverse_False_anneal_True_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_our_VAE_K_5_N_1_learnreverse_True_anneal_False_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_our_VAE_K_5_N_1_learnreverse_True_anneal_True_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_our_VAE_K_5_N_1_learnreverse_True_anneal_True_lrdec_3e-05_lrenc_0.001.pt',
# 'best_model_Multi_our_VAE_K_5_N_5_learnreverse_False_anneal_False_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_our_VAE_K_5_N_5_learnreverse_False_anneal_True_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_our_VAE_K_5_N_5_learnreverse_True_anneal_False_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_our_VAE_K_5_N_5_learnreverse_True_anneal_True_lrdec_0.001_lrenc_None.pt',
# ]

In [5]:
def load_model(path, device):
    device_zero = torch.tensor(0., device=device, dtype=torch.float32)
    device_one = torch.tensor(1., device=device, dtype=torch.float32)
    std_normal = torch.distributions.Normal(loc=device_zero,
                                        scale=device_one)
    uniform = torch.distributions.Uniform(low=device_zero, high=device_one)
    torch_log_2 = torch.tensor(np.log(2), device=device, dtype=torch.float32)
    
    model = torch.load(path, map_location=device)
    
    model.eval()
    model.std_normal = std_normal
    if hasattr(model, 'torch_log_2'):
        model.torch_log_2 = torch_log_2
    if hasattr(model, 'transitions'):
        model.transitions = model.transitions.to(device)
        if type(model.transitions) == type(torch.nn.ModuleList([])):
            for i in range(len(model.transitions)):
                model.transitions[i].device = device
                model.transitions[i].device_zero = device_zero
                model.transitions[i].device_one = device_one
        else:
            model.transitions.device = device
            model.transitions.device_zero = device_zero
            model.transitions.device_one = device_one
    if hasattr(model, 'reverse_kernel'):
        model.reverse_kernel = model.reverse_kernel.to(device)
        model.reverse_kernel.device = device
        model.device_one = device_one
        
    return model

In [6]:
resulting_dict = {'model': [], 'ndcg': [], 'recall': [], 'noise': []}

for noise in [1., 0.]:
    for m in tqdm(models):
        model = load_model(os.path.join('./models', m), device)

        ndcg = []
        recall = []
        for bnum, batch_val in enumerate(dataset.next_val_batch()):
            out = model(batch_val[0], is_training_ph=noise)  ### Here we are using noise when estimate metrics
            pred_val = out[0]
            X = batch_val[0].cpu().detach().numpy()
            pred_val = pred_val.cpu().detach().numpy()
            # exclude examples from training and validation (if any)
            pred_val[X.nonzero()] = -np.inf
            ndcg.append(NDCG_binary_at_k_batch(pred_val, batch_val[1]))
            recall.append(Recall_at_k_batch(pred_val, batch_val[1]))

        ndcg_dist = np.concatenate(ndcg)
        recall_dist = np.concatenate(recall)
        current_ndcg = ndcg_dist.mean()
        current_recall = recall_dist.mean()
        resulting_dict['model'].append(m)
        resulting_dict['ndcg'].append(current_ndcg)
        resulting_dict['recall'].append(current_recall)
        resulting_dict['noise'].append(noise)
        print(m, 'NDCG: ', current_ndcg, '\t', 'Recall: ', current_recall)
        del model

  1%|          | 1/105 [00:04<08:13,  4.75s/it]

best_model_MultiHoffmanVAE_K_1_N_1_learnreverse_False_anneal_False_lrdec_0.001_lrenc_None.pt NDCG:  0.3608305594786048 	 Recall:  0.5907901029535558


  2%|▏         | 2/105 [00:09<08:04,  4.70s/it]

best_model_MultiHoffmanVAE_K_1_N_1_learnreverse_False_anneal_True_lrdec_0.001_lrenc_None.pt NDCG:  0.39038477051597703 	 Recall:  0.6282061284752901


  3%|▎         | 3/105 [00:14<08:11,  4.82s/it]

...

best_model_Multi_our_VAE_K_5_N_5_learnreverse_False_anneal_True_lrdec_0.001_lrenc_None.pt NDCG:  0.4330819446942651 	 Recall:  0.664320280604526


 99%|█████████▉| 104/105 [09:17<00:07,  7.54s/it]

best_model_Multi_our_VAE_K_5_N_5_learnreverse_True_anneal_False_lrdec_0.001_lrenc_None.pt NDCG:  0.39383583561551944 	 Recall:  0.6257787363456164


100%|██████████| 105/105 [09:26<00:00,  5.39s/it]

best_model_Multi_our_VAE_K_5_N_5_learnreverse_True_anneal_True_lrdec_0.001_lrenc_None.pt NDCG:  0.4330531436524725 	 Recall:  0.6635254508746118


In [7]:
pd.set_option('display.max_colwidth', 100)

In [8]:
df = pd.DataFrame(resulting_dict)
df.to_csv('./eval_results.csv', index=False)
df

,model,ndcg,recall,noise
0,best_model_MultiHoffmanVAE_K_1_N_1_learnreverse_False_anneal_False_lrdec_0.001_lrenc_None.pt,0.360831,0.590790,1.0
1,best_model_MultiHoffmanVAE_K_1_N_1_learnreverse_False_anneal_True_lrdec_0.001_lrenc_None.pt,0.390385,0.628206,1.0
2,best_model_MultiHoffmanVAE_K_2_N_2_learnreverse_False_anneal_False_lrdec_0.001_lrenc_None.pt,0.355096,0.584284,1.0
3,best_model_MultiHoffmanVAE_K_2_N_2_learnreverse_False_anneal_True_lrdec_0.001_lrenc_None.pt,0.389430,0.622826,1.0
4,best_model_MultiHoffmanVAE_K_3_N_1_learnreverse_False_anneal_False_lrdec_0.0003_lrenc_0.001.pt,0.364308,0.593124,1.0
5,best_model_MultiHoffmanVAE_K_3_N_1_learnreverse_False_anneal_False_lrdec_3e-05_lrenc_0.001.pt,0.353513,0.576864,1.0
6,best_model_MultiHoffmanVAE_K_3_N_1_learnreverse_False_anneal_True_lrdec_0.0003_lrenc_0.001.pt,0.390911,0.624595,1.0
7,best_model_MultiHoffmanVAE_K_3_N_1_learnreverse_False_anneal_True_lrdec_3e-05_lrenc_0.001.pt,0.379718,0.616490,1.0
8,best_model_MultiHoffmanVAE_K_3_N_3_learnreverse_False_anneal_False_lrdec_0.001_lrenc_None.pt,0.349693,0.575716,1.0
9,best_model_MultiHoffmanVAE_K_3_N_3_learnreverse_False_anneal_True_lrdec_0.001_lrenc_None.pt,0.383247,0.618050,1.0


In [9]:
df[df.noise == 1.].sort_values('ndcg', ascending=False).head(30)

,model,ndcg,recall,noise
98,best_model_Multi_our_VAE_K_5_N_1_learnreverse_False_anneal_True_lrdec_0.001_lrenc_None.pt,0.394348,0.627697,1.0
40,best_model_Multi_ourHoffman_VAE_K_2_N_2_learnreverse_True_anneal_True_lrdec_0.001_lrenc_None.pt,0.393840,0.629602,1.0
100,best_model_Multi_our_VAE_K_5_N_1_learnreverse_True_anneal_True_lrdec_0.001_lrenc_None.pt,0.393834,0.627677,1.0
56,best_model_Multi_ourHoffman_VAE_K_5_N_1_learnreverse_True_anneal_True_lrdec_0.001_lrenc_None.pt,0.393602,0.630550,1.0
52,best_model_Multi_ourHoffman_VAE_K_3_N_3_learnreverse_True_anneal_True_lrdec_0.001_lrenc_None.pt,0.393370,0.627757,1.0
18,best_model_Multi_ourHoffman_VAE_K_1_N_1_learnreverse_False_anneal_True_lrdec_0.001_lrenc_None.pt,0.393153,0.628258,1.0
96,best_model_Multi_our_VAE_K_3_N_3_learnreverse_True_anneal_True_lrdec_0.001_lrenc_None.pt,0.393080,0.628729,1.0
47,best_model_Multi_ourHoffman_VAE_K_3_N_1_learnreverse_True_anneal_True_lrdec_0.001_lrenc_None.pt,0.392986,0.629117,1.0
91,best_model_Multi_our_VAE_K_3_N_1_learnreverse_True_anneal_True_lrdec_0.001_lrenc_None.pt,0.392841,0.629244,1.0
62,best_model_Multi_our_VAE_K_1_N_1_learnreverse_False_anneal_True_lrdec_0.001_lrenc_None.pt,0.392800,0.628058,1.0


In [10]:
df[df.noise == 0.].sort_values('ndcg', ascending=False).head(33)

,model,ndcg,recall,noise
151,best_model_Multi_ourHoffman_VAE_K_3_N_1_learnreverse_True_anneal_True_lrdec_0.0003_lrenc_0.001.pt,0.436991,0.666625,0.0
195,best_model_Multi_our_VAE_K_3_N_1_learnreverse_True_anneal_True_lrdec_0.0003_lrenc_0.001.pt,0.436559,0.665753,0.0
111,best_model_MultiHoffmanVAE_K_3_N_1_learnreverse_False_anneal_True_lrdec_0.0003_lrenc_0.001.pt,0.435024,0.665328,0.0
205,best_model_Multi_our_VAE_K_5_N_1_learnreverse_True_anneal_True_lrdec_0.001_lrenc_None.pt,0.434075,0.663373,0.0
129,best_model_Multi_ourHoffman_VAE_K_1_N_2_learnreverse_True_anneal_True_lrdec_0.001_lrenc_None.pt,0.433890,0.664852,0.0
157,best_model_Multi_ourHoffman_VAE_K_3_N_3_learnreverse_True_anneal_True_lrdec_0.001_lrenc_None.pt,0.433735,0.662477,0.0
169,best_model_Multi_our_VAE_K_1_N_1_learnreverse_True_anneal_True_lrdec_0.001_lrenc_None.pt,0.433503,0.664620,0.0
155,best_model_Multi_ourHoffman_VAE_K_3_N_3_learnreverse_False_anneal_True_lrdec_0.001_lrenc_None.pt,0.433488,0.663795,0.0
175,best_model_Multi_our_VAE_K_1_N_3_learnreverse_False_anneal_True_lrdec_0.001_lrenc_None.pt,0.433231,0.663712,0.0
207,best_model_Multi_our_VAE_K_5_N_5_learnreverse_False_anneal_True_lrdec_0.001_lrenc_None.pt,0.433082,0.664320,0.0


# Parameters tuning

In [4]:
args.metric = NDCG_binary_at_k_batch
args.n_epoches = 50

In [5]:
for lr in [0.01, 0.001, 0.0001]:
    print('lr ', lr)
    model = load_model('./models/best_model_MultiVAE_K_None_N_None_learnreverse_False_anneal_True.pt', 'cuda')
    args.learning_rate = lr

    for p in model.parameters():
        p.requires_grad_(False)
    model.eval()
    fixed_decoder = model.decoder
    del model

    layers = [200, 600, dataset.n_items]

    model = MultiVAE(layers, args=args)
    model.decoder = fixed_decoder
    model.to(device)
    train_model(model, dataset, args)

lr  0.01
Sequential(
  (0): Linear(in_features=20108, out_features=600, bias=True)
  (1): Tanh()
  (2): Linear(in_features=600, out_features=400, bias=True)
)
Sequential(
  (0): Linear(in_features=200, out_features=600, bias=True)
  (1): Tanh()
  (2): Linear(in_features=600, out_features=20108, bias=True)
)


  0%|          | 0/50 [00:00<?, ?it/s]

763.05676
493.5768
484.29013


  2%|▏         | 1/50 [00:10<08:28, 10.38s/it]

Best NDCG: 0.41631043509731697
Current NDCG: 0.41631043509731697
463.54306
482.39844
510.4981


...

Best NDCG: 0.4297082162678262
Current NDCG: 0.4297082162678262
435.4895
418.33298
521.2349


 98%|█████████▊| 49/50 [07:54<00:09,  9.55s/it]

Best NDCG: 0.4297082162678262
Current NDCG: 0.4293932150163262
504.02423
381.1876
517.6016


100%|██████████| 50/50 [08:03<00:00,  9.68s/it]

Best NDCG: 0.4298818419895272
Current NDCG: 0.4298818419895272
